In [1]:
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

#current_date = pd.Timestamp('2023-01-12 12:00:00')
current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our pipeline.
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2024-01-13 10:00:00')


In [3]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [4]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

File 2022-12 was already in local storage
File 2023-01 was already in local storage


In [5]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:02<00:00, 109.08it/s]


In [6]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [7]:
ts_data

,pickup_hour,rides,pickup_location_id,pickup_ts
0,2023-12-16 10:00:00+00:00,0,1,1702720800000
1,2023-12-16 11:00:00+00:00,2,1,1702724400000
2,2023-12-16 12:00:00+00:00,2,1,1702728000000
3,2023-12-16 13:00:00+00:00,0,1,1702731600000
4,2023-12-16 14:00:00+00:00,2,1,1702735200000
...,...,...,...,...
178075,2024-01-13 05:00:00+00:00,1,265,1705122000000
178076,2024-01-13 06:00:00+00:00,1,265,1705125600000
178077,2024-01-13 07:00:00+00:00,1,265,1705129200000
178078,2024-01-13 08:00:00+00:00,2,265,1705132800000


In [8]:
location_ids = ts_data['pickup_location_id'].unique()
len(location_ids)

265

In [9]:
import hopsworks
import src.config as config
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value = config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_METADATA.name,
    version=config.FEATURE_GROUP_METADATA.version,
    description="Time-series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/79009
Connected. Call `.close()` to terminate connection gracefully.


In [10]:
feature_group.insert(ts_data, write_options={"wait_for_job":False})

Uploading Dataframe: 0.00% |          | Rows 0/178080 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/79009/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc7544b5be0>, None)